In [1]:
import pandas as pd

Greedy

In [2]:
with open("controllers/moose_greedy/results/greedy_performance.csv", "r") as f:
    lines = f.read().splitlines()

rows = [line.strip('"').split(',') for line in lines]
columns = rows[0]
data = rows[1:]

import pandas as pd
greedy = pd.DataFrame(data, columns=columns)

# Converta os dados numéricos
for col in ["Distance", "Distance traveled", "Time", "Avg Speed"]:
    greedy[col] = pd.to_numeric(greedy[col], errors='coerce')

print(greedy.head())
print(greedy["Result"].value_counts())


   Distance  Distance traveled    Time  Avg Speed   Result
0     88.08              90.89  120.58       1.14  Success
1     39.36              41.40   56.78       1.36  Success
2    148.35             160.47  153.36       1.06  Success
3    121.34             125.18  130.37       1.41  Success
4    133.20             139.18  156.91       1.06  Success
Result
Success    80
Failure    20
Name: count, dtype: int64


In [3]:
success_rate = (greedy["Result"] == "Success").mean()
print(f"Taxa de sucesso: {success_rate:.2%}")


Taxa de sucesso: 80.00%


In [4]:
avg_time = greedy["Time"].mean()
print(f"Tempo médio: {avg_time:.2f} segundos")

Tempo médio: 117.41 segundos


In [5]:
greedy["Desvio"] = greedy["Distance traveled"] - greedy["Distance"]
print("Desvio médio (Distância percorrida - planejada):")
print(greedy["Desvio"].describe())

Desvio médio (Distância percorrida - planejada):
count    100.00000
mean      -0.96450
std       20.49203
min      -76.45000
25%        0.95500
50%        3.90000
75%        8.75250
max       37.00000
Name: Desvio, dtype: float64


In [6]:
avg_speed = greedy["Avg Speed"].mean()
print(f"Velocidade média: {avg_speed:.2f} m/s")


Velocidade média: 1.01 m/s


In [7]:
grouped = greedy.groupby("Result").agg({
    "Time": ["mean", "std"],
    "Distance traveled": ["mean", "std"],
    "Avg Speed": ["mean", "std"],
    "Desvio": ["mean", "std"]
})
print("Comparação entre Resultados:")
print(grouped)

Comparação entre Resultados:
             Time            Distance traveled            Avg Speed            \
             mean        std              mean        std      mean       std   
Result                                                                          
Failure   78.5380  68.463487          45.27450  39.502238  0.819000  0.149275   
Success  127.1335  59.401562          96.71825  54.999510  1.059125  0.237093   

           Desvio             
             mean        std  
Result                        
Failure -32.80350  25.267935  
Success   6.99525   7.202722  


In [8]:
corr = greedy["Time"].corr(greedy["Distance traveled"])
print(f"Correlação entre tempo e distância percorrida: {corr:.2f}")


Correlação entre tempo e distância percorrida: 0.90


In [9]:
sucessos = greedy[greedy['Result'] == 'Success']
sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100
eficiencia_media_sucesso = sucessos['Efficiency (%)'].mean()

print(f"Eficiência média (sucesso): {eficiencia_media_sucesso:.2f}%")


Eficiência média (sucesso): 96.55%


C:\Users\isabe\AppData\Local\Temp\ipykernel_22224\179856094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100


A*

In [10]:
a_star = pd.read_csv("controllers/moose_astar_supervisor/logs/performance.csv")
a_star.head()

,Distance,Distance traveled,Time,Avg Speed,Battery Used,Height Difference,Completion
0,134.063418,138.036580,371.217591,0.684124,0.834939,5.491311,True
1,64.381674,65.225397,158.566742,0.765357,0.076002,3.887387,True
2,98.954535,105.941125,365.362667,0.815191,0.786068,5.505601,True
3,129.402473,130.279221,244.967768,1.109709,0.871576,6.375358,True
4,82.292162,88.911688,244.983774,0.728626,0.000000,6.470730,True


In [11]:
a_star["Completion"].value_counts()

Completion
False    54
True     46
Name: count, dtype: int64

In [12]:
success_rate = a_star["Completion"].mean()
print(f"Taxa de sucesso: {success_rate:.2%}")

Taxa de sucesso: 46.00%


In [13]:
avg_time = greedy["Time"].mean()
print(f"Tempo médio: {avg_time:.2f} segundos")

Tempo médio: 117.41 segundos


In [14]:
a_star["Desvio"] = a_star["Distance traveled"] - a_star["Distance"]
print("Desvio médio (Distância percorrida - planejada):")
print(a_star["Desvio"].describe())

Desvio médio (Distância percorrida - planejada):
count    100.000000
mean     -40.084466
std       54.724086
min     -183.114938
25%      -85.057155
50%      -13.513878
75%        3.583287
max       16.679392
Name: Desvio, dtype: float64


In [15]:
avg_speed = a_star["Avg Speed"].mean()
print(f"Velocidade média: {avg_speed:.2f} m/s")

Velocidade média: 0.63 m/s


In [16]:
grouped = a_star.groupby("Completion").agg({
    "Time": ["mean", "std"],
    "Distance traveled": ["mean", "std"],
    "Avg Speed": ["mean", "std"],
    "Height Difference": ["mean", "std"]
})

print("Comparação entre Resultados:")
print(grouped)


Comparação entre Resultados:
                  Time             Distance traveled            Avg Speed  \
                  mean         std              mean        std      mean   
Completion                                                                  
False       150.120974  183.903201         24.704365  39.372379  0.625599   
True        315.102704  240.200613         95.138724  47.874324  0.640872   

                     Height Difference            
                 std              mean       std  
Completion                                        
False       0.236304          1.402802  1.998861  
True        0.213160          4.660505  2.663829  


In [17]:
# Filtra só as linhas de sucesso
sucessos = a_star[a_star['Completion'] == True]

# Calcula a eficiência média
sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100
eficiencia_media_sucesso = sucessos['Efficiency (%)'].mean()

print(f"Eficiência média (sucesso): {eficiencia_media_sucesso:.2f}%")


Eficiência média (sucesso): 94.66%


C:\Users\isabe\AppData\Local\Temp\ipykernel_22224\1887011073.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucessos['Efficiency (%)'] = (sucessos['Distance'] / sucessos['Distance traveled']) * 100
